In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import sklearn
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('../input/train.csv', index_col='id')
unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}

In [3]:
scaler = StandardScaler()
labels = train['type']
train = train.drop(columns=['fiberID', 'type'])

In [4]:
_mat = scaler.fit_transform(train)
train = pd.DataFrame(_mat, columns=train.columns, index=train.index)

In [5]:
train_x = train
train_y = labels.replace(label_dict)

In [6]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0003,
)

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(len(train_x.columns)),
  tf.keras.layers.Dense(256*4, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(256*4, activation='elu'),
  tf.keras.layers.Dense(256*3, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(256*3, activation='elu'),
  tf.keras.layers.Dense(256*2, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(256*2, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(256*1, activation='elu'),
  tf.keras.layers.Dense(256*1, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation='elu'),
  tf.keras.layers.Dense(32, activation='elu'),
  tf.keras.layers.Dense(19, activation='softmax')
])

W0212 10:41:06.681532  3832 deprecation.py:506] From C:\Users\TBI\Miniconda3\envs\tf2\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(train_x,
          train_y,
          batch_size=256*3,
          validation_split=0.1,
          epochs=200)

Train on 179991 samples, validate on 20000 samples
Epoch 1/200


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(768, 20), b.shape=(20, 1024), m=768, n=1024, k=20
	 [[{{node dense/MatMul}}]]
	 [[metrics/acc/Identity/_181]]
  (1) Internal: Blas GEMM launch failed : a.shape=(768, 20), b.shape=(20, 1024), m=768, n=1024, k=20
	 [[{{node dense/MatMul}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
test = pd.read_csv('data/test.csv').reset_index(drop=True)
test_ids = test['id']
test = test.drop(columns=['id', 'fiberID'])
test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

In [ ]:
pred_mat = model.predict(test)

In [ ]:
sample = pd.read_csv('data/sample_submission.csv')

submission = pd.DataFrame(pred_mat, index=test.index)
submission = submission.rename(columns=i2lb)
submission = pd.concat([test_ids, submission], axis=1)
submission = submission[sample.columns]
submission.to_csv("submission.csv", index=False)